In [1]:
import pandas as pd
import tabulate
import numpy as np
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm',
'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


In [2]:
def flightNumberImputer(dataFrame, columnName):
    missing = list(dataFrame[dataFrame[columnName].isna() == True].index)
    for i in missing:
        dataFrame.loc[i,columnName] = dataFrame.loc[i-1, columnName] + 10
    dataFrame[columnName] = dataFrame[columnName].astype('int64')
    return dataFrame

In [3]:
flightNumberImputer(df,'FlightNumber')

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


In [4]:
to_from = df.From_To.str.split(pat = '_', expand = True)
to_from.columns = ['From_loc','To_loc']
to_from

,From_loc,To_loc
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


In [5]:
df = pd.concat([to_from,df.iloc[:,1:], ], axis = 1)
df

,From_loc,To_loc,FlightNumber,RecentDelays,Airline
0,LoNDon,paris,10045,"[23, 47]",KLM(!)
1,MAdrid,miLAN,10055,[],<Air France> (12)
2,londON,StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest,PaRis,10075,[13],12. Air France
4,Brussels,londOn,10085,"[67, 32]","""Swiss Air"""


In [6]:
df['From_loc'] = df.apply(lambda x : x['From_loc'].capitalize(), axis = 1)
df['To_loc'] = df.apply(lambda x : x['To_loc'].capitalize(), axis = 1)
df

,From_loc,To_loc,FlightNumber,RecentDelays,Airline
0,London,Paris,10045,"[23, 47]",KLM(!)
1,Madrid,Milan,10055,[],<Air France> (12)
2,London,Stockholm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest,Paris,10075,[13],12. Air France
4,Brussels,London,10085,"[67, 32]","""Swiss Air"""


In [7]:
ncol = []
for i,j in df.iterrows():
    ncol.append(len(j.RecentDelays))
ncol = max(ncol)
ncol

3

In [8]:
colName = []
for i in range(1,ncol+1):
    colName.append('delay_'+str(i))
colName

['delay_1', 'delay_2', 'delay_3']

In [9]:
df.RecentDelays = df.RecentDelays.astype('str')

In [10]:
df.RecentDelays = df.apply(lambda x: x['RecentDelays'][1:-1], axis = 1 )
df

,From_loc,To_loc,FlightNumber,RecentDelays,Airline
0,London,Paris,10045,"23, 47",KLM(!)
1,Madrid,Milan,10055,,<Air France> (12)
2,London,Stockholm,10065,"24, 43, 87",(British Airways. )
3,Budapest,Paris,10075,13,12. Air France
4,Brussels,London,10085,"67, 32","""Swiss Air"""


In [11]:
delay = df.RecentDelays.str.split(",", expand = True)
delay

,0,1,2
0,23,47,None
1,,None,None
2,24,43,87
3,13,None,None
4,67,32,None


In [12]:
delay.rename(columns={0:colName[0],1:colName[1],2:colName[2]}, inplace = True)
delay

,delay_1,delay_2,delay_3
0,23,47,None
1,,None,None
2,24,43,87
3,13,None,None
4,67,32,None


In [13]:
delay = delay.fillna(np.nan)
delay = delay.replace('',np.nan)
delay

,delay_1,delay_2,delay_3
0,23,47,NaN
1,NaN,NaN,NaN
2,24,43,87
3,13,NaN,NaN
4,67,32,NaN


In [14]:
df = pd.concat([df.iloc[:,[0,1,2,4]],delay], axis = 1)
df = df.loc[:,['From_loc','To_loc','FlightNumber','delay_1','delay_2','delay_3']]
df

,From_loc,To_loc,FlightNumber,delay_1,delay_2,delay_3
0,London,Paris,10045,23,47,NaN
1,Madrid,Milan,10055,NaN,NaN,NaN
2,London,Stockholm,10065,24,43,87
3,Budapest,Paris,10075,13,NaN,NaN
4,Brussels,London,10085,67,32,NaN
